# Pydantic

## dataclasses

In [1]:
from dataclasses import dataclass

class Foo:
    bar: int
    baz: str
    
    
# equvivalent to 
class Foo2:
    def __init__(self, bar: int, baz: str):
        self.bar = bar
        self.baz = baz
    
    def __repr__(self):
        return f"Foo2(bar={self.bar}, baz={self.baz})"

In [2]:
foo = Foo(42, "test")
foo

Foo(bar=42, baz='test')

In [3]:
foo2 = Foo2(42, "test")
foo2

Foo2(bar=42, baz=test)

Dataclasses don't check the types.

In [4]:
Foo("wrong", "input")

Foo(bar='wrong', baz='input')

## pydantic

In [ ]:
! pip install pydantic

In [6]:
from pydantic import BaseModel

class PydandicFoo(BaseModel):
    bar: int
    baz: str

In [7]:
foo = PydandicFoo(bar=42, baz="test")
foo

PydandicFoo(bar=42, baz='test')

## Type validation at runtime

In [8]:
foo = PydandicFoo(bar="invalid", baz="input")
foo

ValidationError: 1 validation error for PydandicFoo
bar
  value is not a valid integer (type=type_error.integer)

## Automatic conversions of types if possible

In [9]:
foo = PydandicFoo(bar="42", baz="input")
foo

PydandicFoo(bar=42, baz='input')

## Defaults values

In [10]:
from datetime import datetime
from pydantic import Field

class Order(BaseModel):
    item: str = "Apples"
    ts: datetime = Field(default_factory=datetime.now)

In [11]:
order = Order()
order

Order(item='Apples', ts=datetime.datetime(2022, 5, 6, 11, 54, 41, 754528))

In [12]:
Order()

Order(item='Apples', ts=datetime.datetime(2022, 5, 6, 11, 54, 42, 78755))

## Custom Validators

In [13]:
from pydantic import validator

class UserModel(BaseModel):
    name: str
    username: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

In [14]:
UserModel(name="JakobKogler", username="jakob")

ValidationError: 1 validation error for UserModel
name
  must contain a space (type=value_error)

## Performance

Validating and converting takes time, in deeply nested data or in huge amounts of data, you might want to use the `.construct` method, which just assigns data without validating it. This can be 20x times faster!

In [15]:
UserModel.construct(name="JakobKogler", username="jakob")

UserModel(name='JakobKogler', username='jakob')

## Recursive Models

In [16]:
class Person(BaseModel):
    name: str

class Company(BaseModel):
    company_name: str
    ceo: Person

In [17]:
company = Company(**{"company_name": "Cloudflight", "ceo": {"name": "Roger"}})
company

Company(company_name='Cloudflight', ceo=Person(name='Roger'))

In [18]:
company.ceo.name

'Roger'

## Exporting and importing

In [19]:
company.dict()

{'company_name': 'Cloudflight', 'ceo': {'name': 'Roger'}}

In [20]:
company.dict(exclude={"company_name"}, )

{'ceo': {'name': 'Roger'}}

In [21]:
company.json()

'{"company_name": "Cloudflight", "ceo": {"name": "Roger"}}'

In [22]:
Company.parse_file("company.json")

Company(company_name='Cloudflight', ceo=Person(name='Roger'))

## Static type checking

Mypy can't deal with the autogenerated initialization methods.
Activate the plugin in the `mypy.ini` (or `setup.cfg`) and it works.
https://pydantic-docs.helpmanual.io/mypy_plugin/

## ORM mode

When you have a database (e.g. `SQLAlchemy`) model, and you want to quickly convert it into a Pydantic model.

In [23]:
@dataclass
class CompanyORM:
    company_name: str


class CompanyModel(BaseModel):
    company_name: str

    class Config:
        orm_mode = True

In [24]:
orm = CompanyORM(company_name="Cloudflight")
orm

CompanyORM(company_name='Cloudflight')

In [25]:
company = CompanyModel.from_orm(orm)
company

CompanyModel(company_name='Cloudflight')

## FastAPI

Integration into API Server Framework, to automatically create `openapi.json` and validation errors (status code 422).
Shown at customer project.